<a href="https://colab.research.google.com/github/vincentdavis/zwift_maps/blob/main/zwift_multi_id_event_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use this to map an event recorded at dashboard.vyrtyfy.com with multiple riders**

In [5]:
!rm -r zwift_maps
!git clone https://github.com/vincentdavis/zwift_maps.git

Cloning into 'zwift_maps'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 2), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [57]:
from datetime import datetime
import json
import pandas as pd
import numpy as np

def world_time_to_timestamp(world_time):
    return datetime.fromtimestamp((int(world_time) + 1414016075000) / 1000.0)

def multiple_id_event(json_path, start_time):
  """
  You need to look up the event start time in GMT, UTC.
  This start tiem is used to filter the data. IT is possible the rider,
  never start the event or quits and rider somewhere else, we need to filter these.
  """
  with open(json_path) as j:
    data = json.load(j)
  row_list = []
  for id, id_data in data.items():
    if id_data is not None:
          for row in id_data.values():
            row.update({'id':id})
            row_list.append(row)
  df = pd.DataFrame(row_list)
  df['worldTime'] = df.worldTime.astype(int)
  df['irlTime'] = pd.to_datetime(df['worldTime'], unit='ms') + pd.Timedelta(seconds = (1414016075))
  start_time = datetime.strptime('2021-01-21 19:40', '%Y-%m-%d %H:%M')
  # event_start_time = np.datetime64(start_time)

  # Filters
  df = df[df.irlTime >= event_start_time]
  df = df[df.speed.notnull()]

  df.sort_values(by=['irlTime'], inplace=True)
  df = df.reset_index(drop=True)
  df['-y'] = -df.y
  df = df[['id', 'distance', 'heading', 'heartrate', 'power', 'progress', 'roadTime', 'speed', 'time', 'timestamp', 'worldTime', 'irlTime', 'x', 'y', 'groupId', 'laps']]
  return df

# Tick Tock course:
data_file = 'zwift_maps/event_records/Tick_Tock-2laps-event-1550562.json'
start_time = datetime.strptime('2021-01-21 19:40', '%Y-%m-%d %H:%M')

df = multiple_id_event(data_file, start_time)

In [ ]:
# Tick Tock course:
data_file = 'zwift_maps/event_records/Tick_Tock-2laps-event-1550562.json'
start_time = datetime.strptime('2021-01-21 19:40', '%Y-%m-%d %H:%M')

df = multiple_id_event(data_file, start_time)

In [58]:
df

,id,distance,heading,heartrate,power,progress,roadTime,speed,time,timestamp,worldTime,irlTime,x,y,groupId,laps
0,16793,10629.0,-2568313,151.0,235.0,40448.0,278751,25520118.0,1013.0,1611258008818,197241930929,2021-01-21 19:40:05.929,-161835.5800,289427.200,NaN,NaN
1,590563,6993.0,7265719,153.0,175.0,65280.0,912256,44801360.0,1298.0,1611258009052,197241932644,2021-01-21 19:40:07.644,-2806.2583,-53446.625,NaN,NaN
2,1219352,7460.0,3926110,102.0,175.0,NaN,27205,43665800.0,777.0,1611258009111,197241932845,2021-01-21 19:40:07.845,16051.9690,-26020.191,NaN,NaN
3,1327350,36288.0,3811470,NaN,119.0,49409.0,489618,33266396.0,5983.0,1611258008693,197241932960,2021-01-21 19:40:07.960,60168.8900,-126080.850,1954694.0,NaN
4,817471,5608.0,420206,113.0,116.0,8448.0,139619,14159035.0,954.0,1611258009166,197241933054,2021-01-21 19:40:08.054,-219261.9400,205504.230,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58816,200957,40344.0,-1981268,115.0,101.0,65280.0,164432,34317152.0,3787.0,1611261949523,197245874535,2021-01-21 20:45:49.535,-52253.8200,502136.620,NaN,2.0
58817,817471,37683.0,2506582,107.0,103.0,65280.0,76255,16475782.0,3309.0,1611261949355,197245874544,2021-01-21 20:45:49.544,31769.4960,441306.300,NaN,2.0
58818,1172145,38429.0,548204,126.0,124.0,65280.0,97619,34766100.0,3426.0,1611261949205,197245874548,2021-01-21 20:45:49.548,32810.2800,498783.340,NaN,2.0
58819,964467,33281.0,2493844,163.0,300.0,59648.0,639177,36723288.0,2877.0,1611261951093,197245875535,2021-01-21 20:45:50.535,93558.8600,157529.640,1912410.0,1.0
